In [1]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.architectures import MLP
from neural_blueprints.config.architectures import MLPConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.config.components.composite.projections.input import TabularInputProjectionConfig
from neural_blueprints.config.components.composite.projections.output import TabularOutputProjectionConfig
from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.preprocess import TabularPreprocessor
from neural_blueprints.datasets import MaskedTabularDataset, TabularSingleLabelDataset

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)
data.head()

2025-12-27 17:10:22,261 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-27 17:10:22,262 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,4,0.145129,2,14,5,7,4,3,2,0.000000,0.0,0.397959,39,1
1,0.287671,4,0.052451,12,16,3,5,1,5,2,0.000000,0.0,0.500000,39,1
2,0.150685,2,0.219649,8,4,3,11,1,5,2,0.000000,0.0,0.397959,39,2
3,0.369863,4,0.100153,16,2,3,7,1,3,2,0.076881,0.0,0.397959,39,2
4,0.013699,0,0.061708,16,2,5,0,4,5,1,0.000000,0.0,0.295918,39,1


### Income Inference Accuracy

In [4]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    normalization="batchnorm1d",
    activation='gelu',
    dropout_p=0.2,
    final_activation=None,
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=[len(dataset.cardinalities)*16],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2
    ),
    output_projection=TabularOutputProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        output_cardinalities=[2],
        input_dim=[len(dataset.cardinalities)*16],
        hidden_dims=[len(dataset.cardinalities)*8],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-27 17:10:22,435 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection
2025-12-27 17:10:22,437 - neural_blueprints.architectures.mlp - INFO - Using output projection: TabularOutputProjection


Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.2, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropo

MLPConfig(input_dim=224, hidden_dims=[128, 64, 32, 16], output_dim=224, normalization='batchnorm1d', activation='gelu', dropout_p=0.2, final_activation=None, input_projection=TabularInputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], hidden_dims=[64, 32], output_dim=[224], dropout_p=0.2, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], output_cardinalities=[2], input_dim=[224], hidden_dims=[112], activation='gelu', normalization='batchnorm1d', dropout_p=0.2))

In [6]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="cross_entropy",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=5)

2025-12-27 17:10:22,797 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  20%|██        | 1/5 [00:02<00:09,  2.42s/epoch]

Epoch 1/5, Training Loss: 0.7363, Validation Loss: 0.6913


Training Epochs:  40%|████      | 2/5 [00:04<00:07,  2.42s/epoch]

...

Epoch 3/5, Training Loss: 0.6845, Validation Loss: 0.6872


Training Epochs:  80%|████████  | 4/5 [00:09<00:02,  2.41s/epoch]

Epoch 4/5, Training Loss: 0.6824, Validation Loss: 0.6804


Training Epochs: 100%|██████████| 5/5 [00:12<00:00,  2.41s/epoch]
2025-12-27 17:10:34,857 - neural_blueprints.utils.trainer - INFO - Training completed in 12.06 seconds.
2025-12-27 17:10:34,857 - neural_blueprints.utils.trainer - INFO - Best validation loss: 6.8037e-01


Epoch 5/5, Training Loss: 0.6810, Validation Loss: 0.6830


In [7]:
trainer.predict(val_dataset)

2025-12-27 17:10:35,075 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.05 seconds.


Predictions: tensor([1, 1, 1, 1, 1]), 
 Ground Truth: tensor([1, 2, 1, 1, 1])
Prediction Accuracy: 0.8495


np.float64(0.849539406345957)

### Masked Dataset Inference Accuracy

In [8]:
dataset = MaskedTabularDataset(
    data = data,
    discrete_features = discrete_features,
    continuous_features = continuous_features,
    mask_prob=0.35
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [9]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    normalization="batchnorm1d",
    activation='gelu',
    dropout_p=0.2,
    final_activation="relu",
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=[len(dataset.cardinalities) * 16],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.2
    ),
    output_projection=TabularOutputProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        input_dim=[len(dataset.cardinalities)*16],
        hidden_dims=[8],
        activation="gelu",
        normalization="batchnorm1d",
        dropout_p=0.2
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-27 17:10:35,103 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection


2025-12-27 17:10:35,112 - neural_blueprints.architectures.mlp - INFO - Using output projection: TabularOutputProjection


Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.2, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropo

MLPConfig(input_dim=240, hidden_dims=[128, 64, 32, 16], output_dim=240, normalization='batchnorm1d', activation='gelu', dropout_p=0.2, final_activation='relu', input_projection=TabularInputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], hidden_dims=[64, 32], output_dim=[240], dropout_p=0.2, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], output_cardinalities=None, input_dim=[240], hidden_dims=[8], activation='gelu', normalization='batchnorm1d', dropout_p=0.2))

In [10]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="masked_reconstruction",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=5)

2025-12-27 17:10:35,132 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  20%|██        | 1/5 [00:03<00:15,  3.97s/epoch]

Epoch 1/5, Training Loss: 7.2717, Validation Loss: 25.1792


Training Epochs:  40%|████      | 2/5 [00:07<00:11,  3.94s/epoch]

...

Epoch 3/5, Training Loss: 6.2936, Validation Loss: 24.1839


Training Epochs:  80%|████████  | 4/5 [00:15<00:03,  3.93s/epoch]

Epoch 4/5, Training Loss: 6.2338, Validation Loss: 24.0685


Training Epochs: 100%|██████████| 5/5 [00:19<00:00,  3.96s/epoch]
2025-12-27 17:10:54,913 - neural_blueprints.utils.trainer - INFO - Training completed in 19.78 seconds.
2025-12-27 17:10:54,914 - neural_blueprints.utils.trainer - INFO - Best validation loss: 2.3899e+01


Epoch 5/5, Training Loss: 6.2022, Validation Loss: 23.8992


In [11]:
trainer.predict(val_dataset)

2025-12-27 17:10:55,026 - neural_blueprints.utils.inference.predict - INFO - Feature Column 0:
2025-12-27 17:10:55,026 - neural_blueprints.utils.inference.predict - INFO - Predicted attribute values: [0.37087134 0.24869318 0.23346293 0.24031383 0.24474975]
2025-12-27 17:10:55,027 - neural_blueprints.utils.inference.predict - INFO - True attribute values: [0.5890411  0.46575344 0.30136988 0.56164384 0.02739726]
2025-12-27 17:10:55,027 - neural_blueprints.utils.inference.predict - INFO - Accuracy: 0.1893

2025-12-27 17:10:55,027 - neural_blueprints.utils.inference.predict - INFO - Feature Column 1:
2025-12-27 17:10:55,028 - neural_blueprints.utils.inference.predict - WARNING - Warning: Only one class predicted for this feature.
2025-12-27 17:10:55,028 - neural_blueprints.utils.inference.predict - INFO - Predicted attribute values: [4 4 4 4 4]
2025-12-27 17:10:55,028 - neural_blueprints.utils.inference.predict - INFO - True attribute values: [4. 4. 4. 4. 7.]
2025-12-27 17:10:55,028 - neur

{'avg_discrete_accuracy': np.float64(0.6119109019660357),
 'avg_continuous_accuracy': np.float64(0.6522712457350794),
 'overall_avg_accuracy': np.float64(0.6253643498890503)}